In [1]:
import json
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
import pandas as pd
data = pd.read_excel('/content/chatbot.xltx')
data

,label,question,answer
0,carier,Apa yang dimaksud dengan carier?,Carier adalah suatu alat atau tas dengan kapas...
1,carier,Apa yang dimaksud dengan daypack?,Daypack adalah tempat atau alat yang digunakan...
2,carier,Kapasitas daypack berapa saja yang tersedia?,Kapasitas daypack hanya 30liter saja.
3,carier,Kapasitas carier berapa saja yang tersedia?,Untuk kapasitas carier yang tersedia dimulai d...
4,carier,Apa rekomendasi carier untuk perjalanan 1-2 hari?,Rekomendasi carier untuk perjalan 1-2 adalah c...
...,...,...,...
195,persyaratan,Apakah bisa persyaratan yang diserahkan adalah...,Untuk BPKB belum bisa diserahkan sebagai persy...
196,persyaratan,Apakah bisa persyaratan yang diserahkan adalah...,Untuk Sertifikat belum bisa diserahkan sebagai...
197,persyaratan,Apakah bisa persyaratan yang diserahkan adalah...,Untuk Barang antik belum bisa diserahkan sebag...
198,persyaratan,Apakah bisa persyaratan yang diserahkan adalah...,Untuk Handphone bisa diserahkan sebagai persya...


In [3]:
data['label'].value_counts()

Peralatan_lainnya    42
carier               35
tenda                34
peralatan_masak      27
persyaratan          19
trekking             15
penerangan           14
peralatan_tidur      13
peralatan_lainnya     1
Name: label, dtype: int64

In [4]:
encoder= LabelEncoder()
encoder.fit(data['label'])
training_labels = encoder.transform(data['label'])

# tokenizer

In [6]:
# vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(oov_token=oov_token)
tokenizer.fit_on_texts(data.question)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
sequences = tokenizer.texts_to_sequences(data.question)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# fasttext

In [7]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz

--2023-08-26 03:09:23--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.162.108, 108.157.162.120, 108.157.162.35, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.162.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1227018698 (1.1G) [binary/octet-stream]
Saving to: ‘cc.id.300.vec.gz’

cc.id.300.vec.gz    100%[===================>]   1.14G  71.2MB/s    in 11s     

2023-08-26 03:09:34 (110 MB/s) - ‘cc.id.300.vec.gz’ saved [1227018698/1227018698]



In [8]:
!gzip -dk 'cc.id.300.vec.gz'

In [9]:
from numpy import asarray
from numpy import zeros
embeddings_index = dict()
f = open('/content/cc.id.300.vec')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
embedding_matrix = zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 2000000 word vectors.


# model

In [10]:
num_classes = len(data['label'].unique())
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(keras.layers.Conv1D(128, 5, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 300)           60300     
                                                                 
 conv1d (Conv1D)             (None, 16, 128)           192128    
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                        

# training

In [11]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
7/7 [==============================] - 2s 37ms/step - loss: 2.1907 - accuracy: 0.1400
Epoch 2/500
7/7 [==============================] - 0s 13ms/step - loss: 2.1670 - accuracy: 0.1750
Epoch 3/500
7/7 [==============================] - 0s 14ms/step - loss: 2.1389 - accuracy: 0.1800
Epoch 4/500
7/7 [==============================] - 0s 13ms/step - loss: 2.1040 - accuracy: 0.2100
Epoch 5/500
7/7 [==============================] - 0s 13ms/step - loss: 2.0691 - accuracy: 0.2400
Epoch 6/500
7/7 [==============================] - 0s 14ms/step - loss: 2.0292 - accuracy: 0.2600
Epoch 7/500
7/7 [==============================] - 0s 13ms/step - loss: 1.9920 - accuracy: 0.2550
Epoch 8/500
7/7 [==============================] - 0s 14ms/step - loss: 1.9570 - accuracy: 0.3000
Epoch 9/500
7/7 [==============================] - 0s 18ms/step - loss: 1.9149 - accuracy: 0.3800
Epoch 10/500
7/7 [==============================] - 0s 33ms/step - loss: 1.8675 - accuracy: 0.3800
Epoch 11/500
7/7 [=

# prediksi

In [14]:
import re
from difflib import SequenceMatcher
pd.set_option('mode.chained_assignment', None)
while True:
    inp = input()
    if inp.lower() == "exit":
        break

    result = model.predict(pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
    tag = encoder.inverse_transform([np.argmax(result)])
    x = re.search(r"\w+", str(tag))
    # pilih jawaban berdasarkan label yg sama
    response = data[data['label'] == x.group()]
    # buat skor untuk setiap pertanyaan, pertanyaan dg skor tertinggi (asumsi semakin besar skornya maka semakin mirip dg soal yg diberikan user)
    response['skor'] = response.apply(lambda row: SequenceMatcher(None, inp, row['question']).ratio(), axis = 1)
    # pilih skor terbesar (asumsinya skor yg besar artinya pertanyaan yg diajukan user hampir mirip dg soal di dataset)
    tes = response.loc[response['skor'].idxmax()]
    print("jawaban: ", tes.answer)

Apa yang dimaksud dengan carier?
1/1 [==============================] - 0s 499ms/step
jawaban:  Carier adalah suatu alat atau tas dengan kapasitas besar yang berfungsi untuk membawa barang-barang yang dibawa dalam kegiatan pendakian maupun berkemah.
apa itu carier
1/1 [==============================] - 1s 507ms/step
jawaban:  Tergantung dari pemakaiannya dan merk yang ada.
exit
